In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import random
import plotly.express as px
import time
import seaborn as sns
pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 150
pd.set_option('max_columns', None)

In [2]:
pd.set_option('max_columns', None)

In [3]:
infile = open('..//3_k-means_clustering//apartment_data_frame_clustered','rb')
frame = pkl.load(infile)
infile.close()

In [4]:
frame.head()

,city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url,price_of_sqm,cluster
0,Kołobrzeg,kołobrzeski,zachodniopomorskie,54.175917,15.583267,aftermarket,sprzedaz,75.85,2,4,4,1970.0,850000.0,https://gratka.pl/nieruchomosci/mieszkanie-kolobrzeg/ob/25201643,11206.328279,14
1,Bydgoszcz,NaN,kujawsko-pomorskie,53.141840,17.952490,aftermarket,sprzedaz,64.00,3,2,3,2015.0,539000.0,https://gratka.pl/nieruchomosci/mieszkanie-bydgoszcz/ob/26514629,8421.875000,292
2,Wrocław,Stare Miasto,dolnośląskie,51.101884,17.027313,aftermarket,sprzedaz,122.00,4,3,4,1920.0,849000.0,https://gratka.pl/nieruchomosci/mieszkanie-wroclaw-stare-miasto-ul-michala-baluckiego/ob/25652501,6959.016393,397
3,Toruń,NaN,kujawsko-pomorskie,53.026200,18.590500,aftermarket,sprzedaz,63.50,3,4,3,1993.0,459000.0,https://gratka.pl/nieruchomosci/mieszkanie-torun-ul-wodociagowa/ob/26134151,7228.346457,452
4,Swarzędz,poznański,wielkopolskie,52.416900,17.083000,aftermarket,sprzedaz,53.00,3,3,4,1981.0,359000.0,https://gratka.pl/nieruchomosci/mieszkanie-swarzedz/ob/25317957,6773.584906,121


## data preparation for ann

In [5]:
to_drop = ['city','district','voivodeship','localization_y','localization_x','price_of_sqm','url','offer_type']
frame.drop(to_drop, axis=1, inplace=True)

In [6]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49263 entries, 0 to 49262
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   market    49263 non-null  object 
 1   area      49263 non-null  float64
 2   rooms     49263 non-null  int32  
 3   floor     49263 non-null  int32  
 4   floors    49263 non-null  int32  
 5   build_yr  49263 non-null  float64
 6   price     49263 non-null  float64
 7   cluster   49263 non-null  int32  
dtypes: float64(3), int32(4), object(1)
memory usage: 2.3+ MB


In [7]:
frame.market.unique()

array(['aftermarket', 'primary_market'], dtype=object)

In [8]:
frame.rooms.min(), frame.rooms.nunique(), frame.rooms.max()

(1, 8, 8)

In [9]:
frame.floor.min(), frame.floor.nunique(), frame.floor.max()

(0, 31, 30)

In [10]:
frame.floors.min(), frame.floors.nunique(), frame.floors.max()

(0, 31, 30)

In [11]:
frame.build_yr.min(), frame.build_yr.nunique(), frame.build_yr.max()

(1900.0, 125, 2024.0)

In [12]:
frame.cluster.min(), frame.cluster.nunique(), frame.cluster.max()

(0, 600, 599)

In [13]:
for column in ['cluster','rooms','floor','floors']:
    frame[column] = frame[column].astype('str')

## creating dummy variables

In [14]:
frame = pd.get_dummies(frame, drop_first=True)

In [15]:
frame

,area,build_yr,price,market_primary_market,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,floor_1,floor_10,floor_11,floor_12,floor_13,floor_14,floor_15,floor_16,floor_17,floor_18,floor_19,floor_2,floor_20,floor_21,floor_22,floor_23,floor_24,floor_25,floor_26,floor_27,floor_28,floor_29,floor_3,floor_30,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floors_1,floors_10,floors_11,floors_12,floors_13,floors_14,floors_15,floors_16,floors_17,floors_18,floors_19,floors_2,floors_20,floors_21,floors_22,floors_23,floors_24,floors_25,floors_26,floors_27,floors_28,floors_29,floors_3,floors_30,floors_4,floors_5,floors_6,floors_7,floors_8,floors_9,cluster_1,cluster_10,cluster_100,cluster_101,cluster_102,cluster_103,cluster_104,cluster_105,cluster_106,cluster_107,cluster_108,cluster_109,cluster_11,cluster_110,cluster_111,cluster_112,cluster_113,cluster_114,cluster_115,cluster_116,cluster_117,cluster_118,cluster_119,cluster_12,cluster_120,cluster_121,cluster_122,cluster_123,cluster_124,cluster_125,cluster_126,cluster_127,cluster_128,cluster_129,cluster_13,cluster_130,cluster_131,cluster_132,cluster_133,cluster_134,cluster_135,cluster_136,cluster_137,cluster_138,cluster_139,cluster_14,cluster_140,cluster_141,cluster_142,cluster_143,cluster_144,cluster_145,cluster_146,cluster_147,cluster_148,cluster_149,cluster_15,cluster_150,cluster_151,cluster_152,cluster_153,cluster_154,cluster_155,cluster_156,cluster_157,cluster_158,cluster_159,cluster_16,cluster_160,cluster_161,cluster_162,cluster_163,cluster_164,cluster_165,cluster_166,cluster_167,cluster_168,cluster_169,cluster_17,cluster_170,cluster_171,cluster_172,cluster_173,cluster_174,cluster_175,cluster_176,cluster_177,cluster_178,cluster_179,cluster_18,cluster_180,cluster_181,cluster_182,cluster_183,cluster_184,cluster_185,cluster_186,cluster_187,cluster_188,cluster_189,cluster_19,cluster_190,cluster_191,cluster_192,cluster_193,cluster_194,cluster_195,cluster_196,cluster_197,cluster_198,cluster_199,cluster_2,cluster_20,cluster_200,cluster_201,cluster_202,cluster_203,cluster_204,cluster_205,cluster_206,cluster_207,cluster_208,cluster_209,cluster_21,cluster_210,cluster_211,cluster_212,cluster_213,cluster_214,cluster_215,cluster_216,cluster_217,cluster_218,cluster_219,cluster_22,cluster_220,cluster_221,cluster_222,cluster_223,cluster_224,cluster_225,cluster_226,cluster_227,cluster_228,cluster_229,cluster_23,cluster_230,cluster_231,cluster_232,cluster_233,cluster_234,cluster_235,cluster_236,cluster_237,cluster_238,cluster_239,cluster_24,cluster_240,cluster_241,cluster_242,cluster_243,cluster_244,cluster_245,cluster_246,cluster_247,cluster_248,cluster_249,cluster_25,cluster_250,cluster_251,cluster_252,cluster_253,cluster_254,cluster_255,cluster_256,cluster_257,cluster_258,cluster_259,cluster_26,cluster_260,cluster_261,cluster_262,cluster_263,cluster_264,cluster_265,cluster_266,cluster_267,cluster_268,cluster_269,cluster_27,cluster_270,cluster_271,cluster_272,cluster_273,cluster_274,cluster_275,cluster_276,cluster_277,cluster_278,cluster_279,cluster_28,cluster_280,cluster_281,cluster_282,cluster_283,cluster_284,cluster_285,cluster_286,cluster_287,cluster_288,cluster_289,cluster_29,cluster_290,cluster_291,cluster_292,cluster_293,cluster_294,cluster_295,cluster_296,cluster_297,cluster_298,cluster_299,cluster_3,cluster_30,cluster_300,cluster_301,cluster_302,cluster_303,cluster_304,cluster_305,cluster_306,cluster_307,cluster_308,cluster_309,cluster_31,cluster_310,cluster_311,cluster_312,cluster_313,cluster_314,cluster_315,cluster_316,cluster_317,cluster_318,cluster_319,cluster_32,cluster_320,cluster_321,cluster_322,cluster_323,cluster_324,cluster_325,cluster_326,cluster_327,cluster_328,cluster_329,cluster_33,cluster_330,cluster_331,cluster_332,cluster_333,cluster_334,cluster_335,cluster_336,cluster_337,cluster_338,cluster_339,cluster_34,cluster_340,cluster_341,cluster_342,cluster_343,cluster_344,cluster_345,cluster_346,cluster_347,cluster_348,cluster_349,cluster_35,cluster_350,cluster_351,cluster_3

In [16]:
dummy_apartment_frame = frame.drop(index=frame.index[1:], axis=0)

In [17]:
type(dummy_apartment_frame)

pandas.core.frame.DataFrame

In [18]:
dummy_apartment_frame = dummy_apartment_frame.drop(['price'], axis=1)

In [19]:
dummy_apartment_frame

,area,build_yr,market_primary_market,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,floor_1,floor_10,floor_11,floor_12,floor_13,floor_14,floor_15,floor_16,floor_17,floor_18,floor_19,floor_2,floor_20,floor_21,floor_22,floor_23,floor_24,floor_25,floor_26,floor_27,floor_28,floor_29,floor_3,floor_30,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floors_1,floors_10,floors_11,floors_12,floors_13,floors_14,floors_15,floors_16,floors_17,floors_18,floors_19,floors_2,floors_20,floors_21,floors_22,floors_23,floors_24,floors_25,floors_26,floors_27,floors_28,floors_29,floors_3,floors_30,floors_4,floors_5,floors_6,floors_7,floors_8,floors_9,cluster_1,cluster_10,cluster_100,cluster_101,cluster_102,cluster_103,cluster_104,cluster_105,cluster_106,cluster_107,cluster_108,cluster_109,cluster_11,cluster_110,cluster_111,cluster_112,cluster_113,cluster_114,cluster_115,cluster_116,cluster_117,cluster_118,cluster_119,cluster_12,cluster_120,cluster_121,cluster_122,cluster_123,cluster_124,cluster_125,cluster_126,cluster_127,cluster_128,cluster_129,cluster_13,cluster_130,cluster_131,cluster_132,cluster_133,cluster_134,cluster_135,cluster_136,cluster_137,cluster_138,cluster_139,cluster_14,cluster_140,cluster_141,cluster_142,cluster_143,cluster_144,cluster_145,cluster_146,cluster_147,cluster_148,cluster_149,cluster_15,cluster_150,cluster_151,cluster_152,cluster_153,cluster_154,cluster_155,cluster_156,cluster_157,cluster_158,cluster_159,cluster_16,cluster_160,cluster_161,cluster_162,cluster_163,cluster_164,cluster_165,cluster_166,cluster_167,cluster_168,cluster_169,cluster_17,cluster_170,cluster_171,cluster_172,cluster_173,cluster_174,cluster_175,cluster_176,cluster_177,cluster_178,cluster_179,cluster_18,cluster_180,cluster_181,cluster_182,cluster_183,cluster_184,cluster_185,cluster_186,cluster_187,cluster_188,cluster_189,cluster_19,cluster_190,cluster_191,cluster_192,cluster_193,cluster_194,cluster_195,cluster_196,cluster_197,cluster_198,cluster_199,cluster_2,cluster_20,cluster_200,cluster_201,cluster_202,cluster_203,cluster_204,cluster_205,cluster_206,cluster_207,cluster_208,cluster_209,cluster_21,cluster_210,cluster_211,cluster_212,cluster_213,cluster_214,cluster_215,cluster_216,cluster_217,cluster_218,cluster_219,cluster_22,cluster_220,cluster_221,cluster_222,cluster_223,cluster_224,cluster_225,cluster_226,cluster_227,cluster_228,cluster_229,cluster_23,cluster_230,cluster_231,cluster_232,cluster_233,cluster_234,cluster_235,cluster_236,cluster_237,cluster_238,cluster_239,cluster_24,cluster_240,cluster_241,cluster_242,cluster_243,cluster_244,cluster_245,cluster_246,cluster_247,cluster_248,cluster_249,cluster_25,cluster_250,cluster_251,cluster_252,cluster_253,cluster_254,cluster_255,cluster_256,cluster_257,cluster_258,cluster_259,cluster_26,cluster_260,cluster_261,cluster_262,cluster_263,cluster_264,cluster_265,cluster_266,cluster_267,cluster_268,cluster_269,cluster_27,cluster_270,cluster_271,cluster_272,cluster_273,cluster_274,cluster_275,cluster_276,cluster_277,cluster_278,cluster_279,cluster_28,cluster_280,cluster_281,cluster_282,cluster_283,cluster_284,cluster_285,cluster_286,cluster_287,cluster_288,cluster_289,cluster_29,cluster_290,cluster_291,cluster_292,cluster_293,cluster_294,cluster_295,cluster_296,cluster_297,cluster_298,cluster_299,cluster_3,cluster_30,cluster_300,cluster_301,cluster_302,cluster_303,cluster_304,cluster_305,cluster_306,cluster_307,cluster_308,cluster_309,cluster_31,cluster_310,cluster_311,cluster_312,cluster_313,cluster_314,cluster_315,cluster_316,cluster_317,cluster_318,cluster_319,cluster_32,cluster_320,cluster_321,cluster_322,cluster_323,cluster_324,cluster_325,cluster_326,cluster_327,cluster_328,cluster_329,cluster_33,cluster_330,cluster_331,cluster_332,cluster_333,cluster_334,cluster_335,cluster_336,cluster_337,cluster_338,cluster_339,cluster_34,cluster_340,cluster_341,cluster_342,cluster_343,cluster_344,cluster_345,cluster_346,cluster_347,cluster_348,cluster_349,cluster_35,cluster_350,cluster_351,cluster_352,clu

In [20]:
dummy_apartment_frame.replace(dummy_apartment_frame.iloc[0], 0, inplace=True)

In [21]:
dummy_apartment_frame

,area,build_yr,market_primary_market,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,floor_1,floor_10,floor_11,floor_12,floor_13,floor_14,floor_15,floor_16,floor_17,floor_18,floor_19,floor_2,floor_20,floor_21,floor_22,floor_23,floor_24,floor_25,floor_26,floor_27,floor_28,floor_29,floor_3,floor_30,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floors_1,floors_10,floors_11,floors_12,floors_13,floors_14,floors_15,floors_16,floors_17,floors_18,floors_19,floors_2,floors_20,floors_21,floors_22,floors_23,floors_24,floors_25,floors_26,floors_27,floors_28,floors_29,floors_3,floors_30,floors_4,floors_5,floors_6,floors_7,floors_8,floors_9,cluster_1,cluster_10,cluster_100,cluster_101,cluster_102,cluster_103,cluster_104,cluster_105,cluster_106,cluster_107,cluster_108,cluster_109,cluster_11,cluster_110,cluster_111,cluster_112,cluster_113,cluster_114,cluster_115,cluster_116,cluster_117,cluster_118,cluster_119,cluster_12,cluster_120,cluster_121,cluster_122,cluster_123,cluster_124,cluster_125,cluster_126,cluster_127,cluster_128,cluster_129,cluster_13,cluster_130,cluster_131,cluster_132,cluster_133,cluster_134,cluster_135,cluster_136,cluster_137,cluster_138,cluster_139,cluster_14,cluster_140,cluster_141,cluster_142,cluster_143,cluster_144,cluster_145,cluster_146,cluster_147,cluster_148,cluster_149,cluster_15,cluster_150,cluster_151,cluster_152,cluster_153,cluster_154,cluster_155,cluster_156,cluster_157,cluster_158,cluster_159,cluster_16,cluster_160,cluster_161,cluster_162,cluster_163,cluster_164,cluster_165,cluster_166,cluster_167,cluster_168,cluster_169,cluster_17,cluster_170,cluster_171,cluster_172,cluster_173,cluster_174,cluster_175,cluster_176,cluster_177,cluster_178,cluster_179,cluster_18,cluster_180,cluster_181,cluster_182,cluster_183,cluster_184,cluster_185,cluster_186,cluster_187,cluster_188,cluster_189,cluster_19,cluster_190,cluster_191,cluster_192,cluster_193,cluster_194,cluster_195,cluster_196,cluster_197,cluster_198,cluster_199,cluster_2,cluster_20,cluster_200,cluster_201,cluster_202,cluster_203,cluster_204,cluster_205,cluster_206,cluster_207,cluster_208,cluster_209,cluster_21,cluster_210,cluster_211,cluster_212,cluster_213,cluster_214,cluster_215,cluster_216,cluster_217,cluster_218,cluster_219,cluster_22,cluster_220,cluster_221,cluster_222,cluster_223,cluster_224,cluster_225,cluster_226,cluster_227,cluster_228,cluster_229,cluster_23,cluster_230,cluster_231,cluster_232,cluster_233,cluster_234,cluster_235,cluster_236,cluster_237,cluster_238,cluster_239,cluster_24,cluster_240,cluster_241,cluster_242,cluster_243,cluster_244,cluster_245,cluster_246,cluster_247,cluster_248,cluster_249,cluster_25,cluster_250,cluster_251,cluster_252,cluster_253,cluster_254,cluster_255,cluster_256,cluster_257,cluster_258,cluster_259,cluster_26,cluster_260,cluster_261,cluster_262,cluster_263,cluster_264,cluster_265,cluster_266,cluster_267,cluster_268,cluster_269,cluster_27,cluster_270,cluster_271,cluster_272,cluster_273,cluster_274,cluster_275,cluster_276,cluster_277,cluster_278,cluster_279,cluster_28,cluster_280,cluster_281,cluster_282,cluster_283,cluster_284,cluster_285,cluster_286,cluster_287,cluster_288,cluster_289,cluster_29,cluster_290,cluster_291,cluster_292,cluster_293,cluster_294,cluster_295,cluster_296,cluster_297,cluster_298,cluster_299,cluster_3,cluster_30,cluster_300,cluster_301,cluster_302,cluster_303,cluster_304,cluster_305,cluster_306,cluster_307,cluster_308,cluster_309,cluster_31,cluster_310,cluster_311,cluster_312,cluster_313,cluster_314,cluster_315,cluster_316,cluster_317,cluster_318,cluster_319,cluster_32,cluster_320,cluster_321,cluster_322,cluster_323,cluster_324,cluster_325,cluster_326,cluster_327,cluster_328,cluster_329,cluster_33,cluster_330,cluster_331,cluster_332,cluster_333,cluster_334,cluster_335,cluster_336,cluster_337,cluster_338,cluster_339,cluster_34,cluster_340,cluster_341,cluster_342,cluster_343,cluster_344,cluster_345,cluster_346,cluster_347,cluster_348,cluster_349,cluster_35,cluster_350,cluster_351,cluster_352,clu

In [22]:
list(dummy_apartment_frame.columns)

['area',
 'build_yr',
 'market_primary_market',
 'rooms_2',
 'rooms_3',
 'rooms_4',
 'rooms_5',
 'rooms_6',
 'rooms_7',
 'rooms_8',
 'floor_1',
 'floor_10',
 'floor_11',
 'floor_12',
 'floor_13',
 'floor_14',
 'floor_15',
 'floor_16',
 'floor_17',
 'floor_18',
 'floor_19',
 'floor_2',
 'floor_20',
 'floor_21',
 'floor_22',
 'floor_23',
 'floor_24',
 'floor_25',
 'floor_26',
 'floor_27',
 'floor_28',
 'floor_29',
 'floor_3',
 'floor_30',
 'floor_4',
 'floor_5',
 'floor_6',
 'floor_7',
 'floor_8',
 'floor_9',
 'floors_1',
 'floors_10',
 'floors_11',
 'floors_12',
 'floors_13',
 'floors_14',
 'floors_15',
 'floors_16',
 'floors_17',
 'floors_18',
 'floors_19',
 'floors_2',
 'floors_20',
 'floors_21',
 'floors_22',
 'floors_23',
 'floors_24',
 'floors_25',
 'floors_26',
 'floors_27',
 'floors_28',
 'floors_29',
 'floors_3',
 'floors_30',
 'floors_4',
 'floors_5',
 'floors_6',
 'floors_7',
 'floors_8',
 'floors_9',
 'cluster_1',
 'cluster_10',
 'cluster_100',
 'cluster_101',
 'cluster_102',

## saving the frame

In [23]:
outfile = open('dummy_apartment_frame','wb')
pkl.dump(dummy_apartment_frame,outfile)
outfile.close()

## ann test estimation

In [24]:
from tensorflow.keras.models import load_model
import joblib

In [25]:
model = load_model('..//6_ann_estimator//ann_models//ann_model_2022-06-19--11-31')

In [26]:
scaler = joblib.load('..//6_ann_estimator//scaler')

In [27]:
dummy_apartment_frame.market_primary_market = 1
dummy_apartment_frame.area = 50.0
dummy_apartment_frame.rooms_3 = 1
# dummy_apartment_frame.floor_2 = 1
dummy_apartment_frame.floors_5 = 1
dummy_apartment_frame.build_yr = 2005
dummy_apartment_frame.cluster_298 = 1

In [28]:
dummy_apartment_frame

,area,build_yr,market_primary_market,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,floor_1,floor_10,floor_11,floor_12,floor_13,floor_14,floor_15,floor_16,floor_17,floor_18,floor_19,floor_2,floor_20,floor_21,floor_22,floor_23,floor_24,floor_25,floor_26,floor_27,floor_28,floor_29,floor_3,floor_30,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floors_1,floors_10,floors_11,floors_12,floors_13,floors_14,floors_15,floors_16,floors_17,floors_18,floors_19,floors_2,floors_20,floors_21,floors_22,floors_23,floors_24,floors_25,floors_26,floors_27,floors_28,floors_29,floors_3,floors_30,floors_4,floors_5,floors_6,floors_7,floors_8,floors_9,cluster_1,cluster_10,cluster_100,cluster_101,cluster_102,cluster_103,cluster_104,cluster_105,cluster_106,cluster_107,cluster_108,cluster_109,cluster_11,cluster_110,cluster_111,cluster_112,cluster_113,cluster_114,cluster_115,cluster_116,cluster_117,cluster_118,cluster_119,cluster_12,cluster_120,cluster_121,cluster_122,cluster_123,cluster_124,cluster_125,cluster_126,cluster_127,cluster_128,cluster_129,cluster_13,cluster_130,cluster_131,cluster_132,cluster_133,cluster_134,cluster_135,cluster_136,cluster_137,cluster_138,cluster_139,cluster_14,cluster_140,cluster_141,cluster_142,cluster_143,cluster_144,cluster_145,cluster_146,cluster_147,cluster_148,cluster_149,cluster_15,cluster_150,cluster_151,cluster_152,cluster_153,cluster_154,cluster_155,cluster_156,cluster_157,cluster_158,cluster_159,cluster_16,cluster_160,cluster_161,cluster_162,cluster_163,cluster_164,cluster_165,cluster_166,cluster_167,cluster_168,cluster_169,cluster_17,cluster_170,cluster_171,cluster_172,cluster_173,cluster_174,cluster_175,cluster_176,cluster_177,cluster_178,cluster_179,cluster_18,cluster_180,cluster_181,cluster_182,cluster_183,cluster_184,cluster_185,cluster_186,cluster_187,cluster_188,cluster_189,cluster_19,cluster_190,cluster_191,cluster_192,cluster_193,cluster_194,cluster_195,cluster_196,cluster_197,cluster_198,cluster_199,cluster_2,cluster_20,cluster_200,cluster_201,cluster_202,cluster_203,cluster_204,cluster_205,cluster_206,cluster_207,cluster_208,cluster_209,cluster_21,cluster_210,cluster_211,cluster_212,cluster_213,cluster_214,cluster_215,cluster_216,cluster_217,cluster_218,cluster_219,cluster_22,cluster_220,cluster_221,cluster_222,cluster_223,cluster_224,cluster_225,cluster_226,cluster_227,cluster_228,cluster_229,cluster_23,cluster_230,cluster_231,cluster_232,cluster_233,cluster_234,cluster_235,cluster_236,cluster_237,cluster_238,cluster_239,cluster_24,cluster_240,cluster_241,cluster_242,cluster_243,cluster_244,cluster_245,cluster_246,cluster_247,cluster_248,cluster_249,cluster_25,cluster_250,cluster_251,cluster_252,cluster_253,cluster_254,cluster_255,cluster_256,cluster_257,cluster_258,cluster_259,cluster_26,cluster_260,cluster_261,cluster_262,cluster_263,cluster_264,cluster_265,cluster_266,cluster_267,cluster_268,cluster_269,cluster_27,cluster_270,cluster_271,cluster_272,cluster_273,cluster_274,cluster_275,cluster_276,cluster_277,cluster_278,cluster_279,cluster_28,cluster_280,cluster_281,cluster_282,cluster_283,cluster_284,cluster_285,cluster_286,cluster_287,cluster_288,cluster_289,cluster_29,cluster_290,cluster_291,cluster_292,cluster_293,cluster_294,cluster_295,cluster_296,cluster_297,cluster_298,cluster_299,cluster_3,cluster_30,cluster_300,cluster_301,cluster_302,cluster_303,cluster_304,cluster_305,cluster_306,cluster_307,cluster_308,cluster_309,cluster_31,cluster_310,cluster_311,cluster_312,cluster_313,cluster_314,cluster_315,cluster_316,cluster_317,cluster_318,cluster_319,cluster_32,cluster_320,cluster_321,cluster_322,cluster_323,cluster_324,cluster_325,cluster_326,cluster_327,cluster_328,cluster_329,cluster_33,cluster_330,cluster_331,cluster_332,cluster_333,cluster_334,cluster_335,cluster_336,cluster_337,cluster_338,cluster_339,cluster_34,cluster_340,cluster_341,cluster_342,cluster_343,cluster_344,cluster_345,cluster_346,cluster_347,cluster_348,cluster_349,cluster_35,cluster_350,cluster_351,cluster_352,clu

In [29]:
input_data = scaler.transform(dummy_apartment_frame)

In [30]:
model.predict(input_data)

array([[712340.44]], dtype=float32)

## random forest estimation

In [31]:
model_rf = joblib.load('..//5_random_forest_estimator//random_forest_model')

In [32]:
model_rf.predict(input_data)

array([389066.66666667])